In [1]:
import pandas as pd
import MySQLdb
import os, sys, logging, datetime
from pathlib import Path
from botocore.exceptions import ClientError

In [6]:
import pandas as pd
import MySQLdb
import os, sys, logging, datetime
from pathlib import Path
from botocore.exceptions import ClientError

# Provide Jasmine database name as per the client
jasmine_database = "client_115"

# CREATE A LIST OF ALL BATCHES TO BE CREATED (Please keep the batch file size as 500/1000 images/file)
batches_data = [     
    #(batch_id,'IMAGE','file_path','batch_file.csv')
    # (73,'IMAGE',r'D:\Batch_Creation\van_mould/','20250124_Bidcall_IMG_van_moulding_46_523_B1.csv'),
    (74,'IMAGE',r'D:\Batch_Creation\van_mould/','20250124_Bidcall_IMG_van_moulding_46_534_B2.csv')
]

# Provide the model prediction data file used to copy the execution
# execution_file_name = '/home/cgiuser/Gajanand/da_automation_script/copy_execution_to_jasmine/execution_copy_csv_for_new_major_parts.csv'
execution_file_name = r'D:\Batch_Creation\claimgeniushq-da_automation_script_updatedjune23\claimgeniushq-da_automation_script-7067a802da23\model_prediction\output/bidcall_vanmoulding_final_output.csv'

In [7]:
# Database connection details
db_host = os.getenv("MYSQL_DB_HOST", "jasmine-prod-mysql.ctxdtbqgicfe.us-west-2.rds.amazonaws.com")
db_port = os.getenv("MYSQL_DB_PORT", "3306")
db_user = os.getenv("MYSQL_DB_USER", "claimgeniusadmin")
db_pass = os.getenv("MYSQL_DB_PASS", "8z8qvLs78PhE")
db_name = os.getenv("MYSQL_DB_NAME", jasmine_database)

# Batch details
batch_id = 0
batch_type = "IMAGE"
batch_execution_id = 0
batch_claims = 0
batch_images = 0

# Logging setup
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
today = datetime.datetime.today()
handler = logging.FileHandler(today.strftime('%Y%m%d%H%M%S') + '.log')
handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)


# Get today's date for log file naming
today = datetime.datetime.today()

# Ensure the logs directory exists
log_dir = os.path.join(sys.path[0], 'logs')
os.makedirs(log_dir, exist_ok=True)

# Define log file path
today = datetime.datetime.today()
log_file = os.path.join(log_dir, today.strftime('%Y%m%d%H%M%S') + '.log')

# Check if a handler is already present to avoid adding multiple handlers
if not logger.handlers:
    # Create a new file handler for logging
    handler = logging.FileHandler(log_file)
    handler.setLevel(logging.INFO)

    # Define the log format
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    handler.setFormatter(formatter)

    # Create the logger
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.INFO)

    # Add the handler to the logger
    logger.addHandler(handler)

# Add a test log to verify logging is working
logger.info("Logging has been initialized successfully.")

# MySQL connection
mysql_conn = None

def create_mysql_connection():
    global mysql_conn
    try:
        mysql_conn = MySQLdb.connect(
            host=db_host,
            port=int(db_port),
            user=db_user,
            passwd=db_pass,
            db=db_name
        )
        logger.info("Connected to MySQL server on database: " + db_name)
    except Exception as error:
        logger.exception("Failed to connect to MySQL server.")
        sys.exit("Error: Could not connect to MySQL.")

def add_claim_batch_items(batch_id, batch_type, local_file_name):
    try:
        file_name = local_dir_name + local_file_name
        allowed_cols = ["claim_id", "image_id"] if batch_type == "IMAGE" else []
        batch_items = pd.read_csv(file_name, usecols=allowed_cols).drop_duplicates()
        batch_items = batch_items.rename(columns={'image_id': 'claim_image_id'})
        
        global batch_claims, batch_images
        batch_claims = batch_items["claim_id"].nunique()
        batch_images = batch_items["claim_image_id"].count()
        logger.info(f"Batch {batch_id} contains {batch_claims} claims and {batch_images} images")

        with mysql_conn.cursor() as cursor:
            for _, row in batch_items.iterrows():
                cursor.execute("""
                    INSERT INTO claim_batch_items (claim_batch_id, claim_id, claim_image_id)
                    VALUES (%s, %s, %s)
                """, (batch_id, row['claim_id'], row['claim_image_id']))
            mysql_conn.commit()
            logger.info(f"Claim batch items added for batch {batch_id}")

        return "CLAIM_BATCH_ITEMS_ADDED"
    except Exception as error:
        logger.exception("Error adding claim batch items.")
        return "ERROR"

def add_batch_execution(batch_id):
    try:
        with mysql_conn.cursor() as cursor:
            sSQL = """
                INSERT INTO batch_executions (claim_batch_id, name, description, is_completed, is_deleted, pipeline)
                VALUES (%s, 'Iteration', 'Iteration data copied from pipelines', 0, 0, 'gc')
            """
            cursor.execute(sSQL, (batch_id,))
            mysql_conn.commit()
            batch_execution_id = cursor.lastrowid
            logger.info(f"Batch execution id {batch_execution_id} created for batch {batch_id}")
            return batch_execution_id
    except Exception as error:
        logger.exception("Error adding batch execution.")
        return 0

def get_image_executions_from_pipelines(local_file_name, execution_file_name):
    try:
        # Load the data for batch executions
        file_name = local_dir_name + local_file_name
        chunksize = 400
        executions_allowed_cols = ['claim_id', 'image_id', 'model', 'result', 'polygons']
        
        # Load the executions file
        executions = pd.read_csv(execution_file_name, usecols=executions_allowed_cols)
        executions = executions.rename(columns={'image_id': 'claim_image_id'}).fillna('')

        # Ensure data types are correct for merge and insertion
        executions['claim_image_id'] = executions['claim_image_id'].astype('int64')
        executions['claim_id'] = executions['claim_id'].astype('int64')
        executions['model'] = executions['model'].astype(str)
        executions['result'] = executions['result'].astype(str)
        executions['polygons'] = executions['polygons'].astype(str)

        logger.info("Execution data loaded successfully for processing.")

        # Insert data in chunks
        for claim_list in pd.read_csv(file_name, chunksize=chunksize):
            claim_list = claim_list.rename(columns={'image_id': 'claim_image_id'})
            claim_list['claim_image_id'] = claim_list['claim_image_id'].astype('int64')
            claim_list['claim_id'] = claim_list['claim_id'].astype('int64')
            
            # Merge claim list with execution data
            dataset = pd.merge(claim_list[['claim_id', 'claim_image_id']], executions, on=['claim_id', 'claim_image_id'])
            dataset['batch_execution_id'] = batch_execution_id
            dataset = dataset[['batch_execution_id', 'claim_id', 'claim_image_id', 'model', 'result', 'polygons']]

            # Log dataset information before attempting to insert
            if dataset.empty:
                logger.warning(f"No matching data found for batch {batch_id} in this chunk.")
                continue
            else:
                logger.info(f"Preparing to insert {len(dataset)} records into batch_execution_results.")

            # Insert records into MySQL
            with mysql_conn.cursor() as cursor:
                print((f"Dataset columns: {dataset.columns.tolist()}"))
                for _, row in dataset.iterrows():
                    print("Inserting records to batch_execution_results")
                    cursor.execute("""
                        INSERT INTO batch_execution_results (batch_execution_id, claim_id, claim_image_id, model, result, polygons)
                        VALUES (%s, %s, %s, %s, %s, %s)
                    """, (row['batch_execution_id'], row['claim_id'], row['claim_image_id'], row['model'], row['result'], row['polygons']))
            mysql_conn.commit()
            logger.info(f"Chunk with {len(dataset)} executions copied successfully to batch_execution_results.")
            print(f"Chunk with {len(dataset)} executions copied successfully to batch_execution_results.")
            
    except Exception as error:
        logger.exception("Error during batch execution insertion.")


def copy_executions_from_pipelines_to_jasmine():
    global batch_execution_id
    try:
        batch_items_result = add_claim_batch_items(batch_id, batch_type, s3_object_name)
        if batch_items_result == "CLAIM_BATCH_ITEMS_ADDED":
            batch_execution_id = add_batch_execution(batch_id)
            if batch_execution_id != 0 and batch_type == "IMAGE":
                get_image_executions_from_pipelines(s3_object_name, execution_file_name)
    except Exception as error:
        logger.exception("Error in copying executions from pipelines.")
    finally:
        if mysql_conn:
            mysql_conn.close()

# Execution flow
create_mysql_connection()
if batches_data:
    for data in batches_data:
        batch_id, batch_type, local_dir_name, s3_object_name = data
        logger.info(f"Process started at {datetime.datetime.now()} for batch id {batch_id}")
        copy_executions_from_pipelines_to_jasmine()
        logger.info(f"Process completed at {datetime.datetime.now()} for batch id {batch_id}")


Dataset columns: ['batch_execution_id', 'claim_id', 'claim_image_id', 'model', 'result', 'polygons']
Inserting records to batch_execution_results
Inserting records to batch_execution_results
Inserting records to batch_execution_results
Inserting records to batch_execution_results
Inserting records to batch_execution_results
Inserting records to batch_execution_results
Inserting records to batch_execution_results
Inserting records to batch_execution_results
Inserting records to batch_execution_results
Inserting records to batch_execution_results
Inserting records to batch_execution_results
Inserting records to batch_execution_results
Inserting records to batch_execution_results
Inserting records to batch_execution_results
Inserting records to batch_execution_results
Inserting records to batch_execution_results
Inserting records to batch_execution_results
Inserting records to batch_execution_results
Inserting records to batch_execution_results
Inserting records to batch_execution_results

In [ ]:

# # MYSQL CONNECTION DETAILS - JASMINE DB DEV/QA
# mysql_conn      = None
# mysql_db_host   = os.environ.get("MYSQL_DB_HOST", "cg-mysql.ctxdtbqgicfe.us-west-2.rds.amazonaws.com")
# mysql_db_port   = os.environ.get("MYSQL_DB_PORT", "3306")
# mysql_db_user   = os.environ.get("MYSQL_DB_USER", "claimgeniusadmin")
# mysql_db_pass   = os.environ.get("MYSQL_DB_PASS", "8z8qvLs78PhE")
# mysql_db_name   = os.environ.get("MYSQL_DB_NAME", 'client_robin')

# def create_mysql_connection(db_host, db_port, db_user, db_pass, db_name) :
#     try:
#         # Make the connection variable global to be used later
#         global mysql_conn 
#         conn_url = 'mysql://{0}:{1}@{2}:{3}'.format(db_user, db_pass, db_host, db_port)
#         mysql_conn = create_engine(conn_url)
#         if(mysql_conn.connect()):
#             pass
# #             logger.info("Connected to mysql server")
#     except Exception as error:
#         pass
# #         logger.exception("Connection to mysql server failed.")
# #         logger.exception(error)
#         print(error)
        
# create_mysql_connection(mysql_db_host, mysql_db_port, mysql_db_user, mysql_db_pass, mysql_db_name)
# mysql_conn
# # db_conn = mysql_conn.connect()